
# MSc Individual Project: Anomaly Detection for Assisted Independent Living

Author: Christina Spanellis
 
Sections 1-4 of this notebook define the necessary method definitions and constants for this project.

Section 5 contains code to build and test the system

## Sections
### 1. [Data preparation and pre-processing](#section1)
### 2. [Anomalous Data Generation Module](#section2)
### 3. [Prediction Module](#section3)
### 4. [Anomaly Detection Module](#section4)
### 5. [Running the system](#section5)

In [ ]:
# import the necessary packages for the project
from pandas import read_csv
from datetime import datetime
from matplotlib import pyplot
from pandas import DataFrame
from pandas import concat
import pandas
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras import callbacks
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras import models
import keras_tuner as kt
import numpy as np
from keras.metrics import mean_squared_error
from numpy import sqrt

<a id='section1'></a>
## Section 1: Data preparation and pre-processing

Two different data sets had to be cleaned and prepared to be in a suitable format for the prediction module. 

Data set 1: CASAS HH101

Data set 2: CASAS HH102

The below definitions define the constants and logic needed for loading and pre-processing. 

In [ ]:
## CONSTANTS AND GLOBALS DEFINITION

SENSOR_EVENT_WINDOW_SIZE = 19
HH101_PREDICTION_SENSORS = ["M003", "LS002", "M004", "M005", "LS005", "MA015", "M012", "M010"]
HH102_PREDICTION_SENSORS = ["M007", "T105", "LS008", "M004", "M002", "LS010", "MA009", "M018", "LS002"]
NUM_FINAL_FEATURES = 10
train_x = None
NUMBER_IN_FEATURES = None
DATASET = None
NUMBER_PREDICTIONS = None

In [ ]:
# load the hh101 data set
def load_hh101():
    # load the data set
    hh101 = read_csv('datasets/hh101/hh101.csv', names=["Sensor",1,2,"Value","Type"])
    hh101.drop(columns={1,2,"Type"}, inplace=True)
    # replace string values and drop unwanted sensor readings
    hh101 = hh101[hh101["Sensor"].str.contains("BAT") == False]
    # hh101 = hh101[hh101["Sensor"] != "D001"]
    # hh101 = hh101[hh101["Sensor"] != "LS010"]
    hh101["Value"] = hh101["Value"].replace({"ON" : 1.0, "OFF" : 0.0})
    hh101["Value"] = hh101["Value"].replace({"ABSENT" : 1.0, "PRESENT" : 0.0})
    hh101["Value"] = hh101["Value"].replace({"OPEN" : 1.0, "CLOSE" : 0.0})
    # creating a mapping of the sensor names to keep track of them
    count = 0
    hh101_sensor_id_mapping = {}
    for sensor in hh101["Sensor"].values:
        if sensor not in hh101_sensor_id_mapping:
            hh101_sensor_id_mapping[sensor] = count
            count+=1
    hh101_reversed_mapping = {y: x for x, y in hh101_sensor_id_mapping.items()}
    return (hh101, hh101_sensor_id_mapping, hh101_reversed_mapping)

# load the hh102 dataset 
def load_hh102():
    # load the data set
    hh102 = read_csv('datasets/hh102/hh102.csv', names=["Sensor",1,2,"Value","Type"])
    hh102.drop(columns={1,2,"Type"}, inplace=True)
    # replace string values and drop unwanted sensor readings
    hh102 = hh102[hh102["Sensor"].str.contains("BAT") == False]
    # hh102 = hh102[hh102["Sensor"] != "D001"]
    # hh102 = hh102[hh102["Sensor"] != "LS010"]
    hh102["Value"] = hh102["Value"].replace({"ON" : 1.0, "OFF" : 0.0})
    hh102["Value"] = hh102["Value"].replace({"ABSENT" : 1.0, "PRESENT" : 0.0})
    hh102["Value"] = hh102["Value"].replace({"OPEN" : 1.0, "CLOSE" : 0.0})
    # creating a mapping of the sensor names to keep track of them
    count = 0
    hh102_sensor_id_mapping = {}
    for sensor in hh102["Sensor"].values:
        if sensor not in hh102_sensor_id_mapping:
            hh102_sensor_id_mapping[sensor] = count
            count+=1
    hh102_reversed_mapping = {y: x for x, y in hh102_sensor_id_mapping.items()}
    return (hh102, hh102_sensor_id_mapping, hh102_reversed_mapping)


In [ ]:

def remove_sensors(dataset, sensors, mapping):
    for i in range (len(sensors)):
        sensors[i] = mapping[sensors[i]]
    return dataset.drop(columns=sensors)
     

In [ ]:
from sklearn.decomposition import PCA
def pca_decomp(dataset, reversed_mapping, name):
    dataset = dataset.copy()
    min_max_scaler = MinMaxScaler()
    dataset[dataset.columns] = min_max_scaler.fit_transform(dataset)
    pca = PCA(n_components=20)
    components = pca.fit_transform(dataset)
    n_pcs= pca.n_components_ # get number of component
    # get the index of the most important feature on EACH component
    most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]
    initial_feature_names = dataset.columns
    # get the most important feature names
    most_important_features = [initial_feature_names[most_important[i]] for i in range(n_pcs)]
    most_important_features = [i for n, i in enumerate(most_important_features) if i not in most_important_features[:n]] 
    for i in range(NUM_FINAL_FEATURES):
        most_important_features[i] = reversed_mapping[most_important_features[i]]
    features = range(pca.n_components_)
    _ = plt.figure(figsize=(15, 5))
    _ = plt.bar(features, pca.explained_variance_)
    _ = plt.xlabel('PCA feature')
    _ = plt.ylabel('Variance')
    _ = plt.xticks(features)
    _ = plt.title("Importance of the Principal Components based on inertia")
    plt.savefig("data_plots/"+name +"/pca")
    plt.close()
    most_important_features = most_important_features[:NUM_FINAL_FEATURES]
    return most_important_features



In [ ]:
# def check_stationary_and_autocorrelated():
    

This method transforms the data set into a format where the columns represent the various sensor values and segment the data into 20 event sensor windows.

This means that each row in the data set represents the activations for the previous 20 sensor event activations

In [ ]:
def transform_data(dataset, sensor_id_mapping):
    data = []
    starting_date_time = datetime.strptime(dataset.index[0], '%Y-%m-%d %H:%M:%S.%f')
    starting_date_time = starting_date_time.replace(microsecond=0)
    sensor_vals = [0.0] * len(sensor_id_mapping)
    event_count = 0 ## counter used to segment the data into sensor event windows
    for i, row in dataset.iterrows():
        curr_date_time =  datetime.strptime(i, '%Y-%m-%d %H:%M:%S.%f')
        curr_date_time = curr_date_time.replace(microsecond=0)
        if (event_count >= SENSOR_EVENT_WINDOW_SIZE):
            values = [starting_date_time.strftime("%m-%d-%Y %H:%M:%S")]
            values.extend(sensor_vals)
            data.append(values)
            starting_date_time = curr_date_time
            sensor_vals = [0.0] * len(sensor_id_mapping)
            event_count = 0
        event_count +=1
        if "D" in row["Sensor"] or "M" in row["Sensor"]:
            if sensor_vals[sensor_id_mapping[row["Sensor"]]] == 0:
                sensor_vals[sensor_id_mapping[row["Sensor"]]] = 1
            else:
                sensor_vals[sensor_id_mapping[row["Sensor"]]] += 1
        else:
                sensor_vals[sensor_id_mapping[row["Sensor"]]] = row["Value"]

    columns = [i for i in range (0,len(sensor_id_mapping))]
    final_columns = ["Time"]
    final_columns.extend(columns)
    # set the index of the dataframe to be the time column
    new_data = pandas.DataFrame.from_records(data, columns=final_columns)
    new_data["Time"] = pandas.to_datetime(new_data["Time"], format='%m-%d-%Y %H:%M:%S')
    new_data = new_data.set_index("Time")
    return new_data   


This method plots all the data (non-normalised) after formatting

In [392]:
def plot_cleaned_data(data, reversed_mapping, name, path):
    changed_legend = data.rename(columns = reversed_mapping)
    ax = changed_legend.plot(subplots=True,figsize=(12,24), sharey=True, ylabel="Sensor Value")
    pyplot.tight_layout()
    pyplot.savefig("data_plots/" + name + path, dpi=1200)

This method transforms the series into a format suitable for a supervised learning problem.

Eg. [Sensor1(t-1), Sensor2(t-1), Sensor1(t), Sensor2(t)]

Where readings at t-1 represent the sensor activations for the period before the activations at t.

Sensor activations at time t then become the ground truth values for activations at t-1 in the prediction module

In [ ]:
def transform_series_to_supervised(new_data, prediction_sensors, important_features, reversed_mapping):
    df = new_data.copy()
    # print(df)
    # print(df)
    # scale values
    values = df.values
    min_max_scaler = MinMaxScaler()
    scaled_values = min_max_scaler.fit_transform(values)
    normalized_df = pandas.DataFrame(scaled_values)
    n_vars = len(normalized_df.columns)
    time_Series = normalized_df.copy().set_index(df.index)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(1, 0, -1):
        sequence = normalized_df.shift(i)
        sequence = sequence.rename(columns=reversed_mapping)
        sequence = sequence[important_features]
        cols.append(sequence)
        names += [('%s(t-%d)' % (label, i)) for label in sequence.columns]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, 1):
        sequence = normalized_df.shift(-i)
        sequence = sequence.rename(columns=reversed_mapping)
        sequence = sequence[prediction_sensors]
        cols.append(sequence.shift(-i))
        names += [('%s(t)' % (label)) for label in sequence.columns]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return (agg, normalized_df)

This method plots the activations for the sensors in the data set, grouped into figures by sensor type.

In [367]:
def plot_normalised_sensor_activations(dataset_name, normalized_df, reversed_mapping):
    changed_legend = normalized_df.rename(columns = reversed_mapping)
    doors =[]
    lights = []
    temp = []
    motion = []
    for key in reversed_mapping:
        if "D" in reversed_mapping[key]:
            doors.append(key)
        elif "L" in reversed_mapping[key]:
            lights.append(key)
        elif "T1" in reversed_mapping[key]:
            temp.append(key)
        else:
            motion.append(key)
    sensors = [doors, lights, temp, motion]
    names = ["Door Sensors", "Light Sensors", "Temperature Sensors", "Motion Sensors"]
    for i, sensor in enumerate(sensors):
        if "Door" or "Temperature" in names[i]:
            figsize = (11,4)
        if ((len(sensor) / 2) % 2 == 0):
            fig, axs = pyplot.subplots(int(len(sensor) / 2),2,  sharex=True, sharey=True, figsize=(11,8))
        else:
            fig, axs = pyplot.subplots(int(len(sensor) / 2)+1,2,  sharex=True, sharey=True, figsize=(11,4))

        count = 0
        for ax in axs.flat:
            if (count < len(sensor)):
                ax.plot(changed_legend[reversed_mapping[sensor[count]]])
                ax.set_title(reversed_mapping[sensor[count]])
                ax.set_xticks([0, 10000, 20000, 30000, 40000, 50000])
                ax.set_yticks([0.0, 0.5, 1.0])
                count += 1
            else:
                fig.delaxes(ax)
        for ax in axs.flat:
            ax.label_outer()
        fig.suptitle(names[i])
        fig.supxlabel("SEW")
        fig.supylabel("Sensor Value")
        pyplot.tight_layout()
        pyplot.savefig("data_plots/" + dataset_name + "/" +names[i],dpi=1200)

Discover the features with the highest correlations

In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt

def find_correlations(dataset, path, name, reversed_mapping):
    min_max_scaler = MinMaxScaler()
    scaled_values = min_max_scaler.fit_transform(dataset.values)
    normalized_df = pandas.DataFrame(scaled_values)
    normalized_df = normalized_df.rename(columns = reversed_mapping)
    corrmat = normalized_df.corr(method='pearson', min_periods=100)
    corrmat = np.abs(corrmat)
    sns.set(context="paper", font="monospace")
    f, ax = plt.subplots(figsize=(12, 9))
    sns.heatmap(corrmat, vmax=0.8, square=True, xticklabels = True, yticklabels = True)
    pyplot.title(name + " Pearson correlation values between sensors (absolute valued).")
    pyplot.xlabel("Sensor ID")
    pyplot.ylabel("Sensor ID")
    pyplot.tight_layout()
    pyplot.savefig("data_plots/" + path + name)
    triangluar_corrmat = np.triu(corrmat, k=1)
    values = np.where(triangluar_corrmat >= 0.6)
    values = list(zip(values[0], values[1]))
    for x in range (len(values)):
        values[x] = (reversed_mapping[values[x][0]], reversed_mapping[values[x][1]], triangluar_corrmat[values[x][0]][values[x][1]])
    return values 
    

The data needs to be separated out into training (70%), testing (20%) and anomalous portions (10%).

The anomalous portion of the data is held back for synthetic anomaly injection to later be used to test the AD system

In [ ]:
def create_train_test_split(dataset):
    values = dataset.values
    train_split = int(0.7 * len(values))
    anomaly_split = int(0.9 * len(values))
    train = values[:train_split, :]
    test = values[train_split:anomaly_split, :]
    anomalies = values[anomaly_split:, :]

    train_x, train_y = train[:, :NUMBER_IN_FEATURES], train[:, NUMBER_IN_FEATURES:]

    test_x, test_y = test[:, :NUMBER_IN_FEATURES], test[:, NUMBER_IN_FEATURES:]

    anomaly_x, anomaly_y = anomalies[:, :NUMBER_IN_FEATURES], anomalies[:, NUMBER_IN_FEATURES:]

    train_x = np.asarray(train_x).astype(np.float32)
    train_y = np.asarray(train_y).astype(np.float32)
    test_y = np.asarray(test_y).astype(np.float32)
    test_x = np.asarray(test_x).astype(np.float32)
    train_x = train_x.reshape((train_x.shape[0], 1, train_x.shape[1]))
    test_x = test_x.reshape((test_x.shape[0], 1, test_x.shape[1]))
    return (train_x, train_y, test_x, test_y, anomaly_x, anomaly_y)


<a id='section3'></a>

## Section 3: Prediction Module

The following section contains the logic to train models for the Prediction Module

A method to save trained models and plot their loss (used for experimentation)

In [ ]:
import pickle

def save_model_and_plot(model, filename, history):
    model.save("best_models/" + filename +"/best_model", history)
    pyplot.plot(history.history['loss'], label='Loss')
    pyplot.plot(history.history['val_loss'], label='Validation Loss')
    pyplot.xlabel("Epochs")
    pyplot.ylabel("Loss")
    pyplot.title("Training and validation loss of final "+ filename.upper() +" model.")
    pyplot.tight_layout()
    pyplot.legend()
    pyplot.savefig("best_models/"+filename.upper()+"/best_model")

This creates a model suitable for hyper parameter tuning in in keras

In [ ]:
def build_model(hp):
    # design network
    model = Sequential()
    model.add(LSTM(hp.Int('input_lstm_layer', min_value = 50, max_value = 500, step = 50), input_shape=(train_x.shape[1], train_x.shape[2]), return_sequences = True))
    for i in range (hp.Int('n_layers', 1, 3)):
            model.add(LSTM(hp.Int(f'hidden_lstm_layer_{i}', min_value = 50, max_value = 500, step = 50), return_sequences = True))
    model.add(LSTM(hp.Int(f'final_lstm_layer', min_value = 50, max_value = 500, step = 50)))
    model.add(Dropout(hp.Float('Dropout_rate',min_value=0,max_value=0.5,step=0.1)))
    model.add(Dense(NUMBER_PREDICTIONS, activation = hp.Choice('dense_activation', values=['relu', 'sigmoid'],default='relu')))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['mse'])
    return model

Create and use the keras tuner

In [ ]:
def tune_model(x_train, y_train, name):

    tuner = kt.BayesianOptimization(
        build_model,
        objective='mse',
        max_trials=20,
        directory="tensorflow/"+name+"/",
        project_name="models",
        overwrite = False
    )
    tuner.search(
        x_train,
        y_train,
        batch_size = 128,
        validation_split=0.2,
        epochs = 500,
        callbacks=[callbacks.TensorBoard(log_dir="/tmp/tb_logs/"+name, histogram_freq=1), callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30), callbacks.ModelCheckpoint('best_model_es.h5',monitor='val_loss',mode='min',save_best_only=True)],

    )
    return tuner

Train new model with best hyper-parameters

In [ ]:
import os.path

def get_final_model(train_x, train_y, test_x, test_y, tuner):
    model = None
    filename = "best_models/" + DATASET + "/best_model"
    if os.path.isdir(filename):
        model = models.load_model(filename)
    else:
        model = train_final_model(train_x, train_y, test_x, test_y, tuner)
    return model

def train_final_model(train_x, train_y, test_x, test_y, tuner):
    model = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
    history = model.fit(train_x, train_y, epochs=1000, validation_data=(test_x, test_y))
    save_model_and_plot(model, DATASET, history)
    return model


In [ ]:
# from numpy import concatenate
# from math import sqrt
# from sklearn.metrics import mean_squared_error


# # make a prediction
# yhat = model.predict(test_x)
# test_x = test_x.reshape((test_x.shape[0], features))
# print(test_x)
# print(yhat)
# # # calculate RMSE
# rmse = sqrt(mean_squared_error(test_y, yhat))
# print('Test RMSE: %.3f' % rmse)


<a id='section2'></a>
## Section 2: Anomamlous Data Generation Module

The following section contains the logic for the Anomalous Data Generation Module

This method takes the test data and inserts anomalies into a specified fraction of the data

In [ ]:
# Try generating some anomalous data
# So decide on the number of anomalous samples.. for example 10
# Then just affect one 20 event sensor window
# One idea is to just scale data to be bigger than 1, in theory this is an anomaly but isn't really in the expected range of input...
# https://github.com/tirthajyoti/Synthetic-data-gen/blob/master/Notebooks/Time%20series%20synthesis%20with%20anomaly.ipynb

 # retrieves the mean, min and max of the different features in the data set
def get_stats(dataset):
    return (dataset.mean(axis=0), dataset.min(axis=0), dataset.max(axis=0))

def generate_anomalous_data(stats, anomaly_x, anomaly_y):
    anomaly_split = 100
    means = stats[0]
    mins = stats[1]
    maxs = stats[2]
    increase_mag = True
    number_anomalies = 10
    # now need to randomly select one row of data to to alter, so as to not alter the underlying sequence
    anomaly_scale = 1.1
    # types of generated anomalies?
    # - random
    # - intentional anomalies
    #   - swap anomalies
    #   - activate/deactivate anomalies
    # 1. random anomalies
    random_anomaly_x = anomaly_x[:anomaly_split, :]
    generate_random_anomaly(anomaly_y[:anomaly_split, :], number_anomalies, anomaly_scale, maxs, mins)

    # 2. intentional anomalies
    # # 2.1. one simple anomaly is all sensor values = 0
    zero_anomaly_x = anomaly_x[:anomaly_split*2, :]
    generate_intentional_anomaly(anomaly_y[anomaly_split:anomaly_split*2, :], number_anomalies, anomaly_scale, maxs, mins, "zero")

    # # 2.2 activate some portion of non-active sensors
    activate_anomaly_x = anomaly_x[:anomaly_split*3, :]
    generate_intentional_anomaly(anomaly_y[anomaly_split*2:anomaly_split*3, :], number_anomalies, anomaly_scale, maxs, mins, "activate")

    deactivate_anomaly_x = anomaly_x[:anomaly_split*4, :]
    # # 2.3 de-activate active, activate portion of non-active
    generate_intentional_anomaly(anomaly_y[anomaly_split*3:anomaly_split*4, :], number_anomalies, anomaly_scale, maxs, mins, "deactivate")

    return None

In [ ]:
def generate_random_anomaly(anomaly_y, number_anomalies, anomaly_scale, maxs, mins):
    row_ids = np.random.choice(anomaly_y.shape[0], size=number_anomalies, replace=False)
    random_anomalous_y = pandas.DataFrame(anomaly_y)
    random_anomalous_y_copy = random_anomalous_y.copy()
    for row_id in row_ids:
        new_data = [0.0] * len(random_anomalous_y.columns)
        for y in range(len(new_data)):
            # if increase_mag:
                new_data[y] = np.random.uniform(low=mins[y], high=anomaly_scale * (maxs[y] - mins[y]))
            # else:
                # random_anomalous_y[row_id][y] = np.random.uniform(low=anomaly_scale * (maxs[y] - mins[y]), high=anomaly_scale * (maxs[y] - mins[y]))
        random_anomalous_y.loc[row_id] = new_data
    plot_anomalsed_data(row_ids, random_anomalous_y, random_anomalous_y_copy, "random_anomaly")

In [ ]:
def generate_intentional_anomaly(anomaly_y, number_anomalies, anomaly_scale, maxs, mins, type):
    row_ids = np.random.choice(anomaly_y.shape[0], size=number_anomalies, replace=False)
    random_anomalous_y = pandas.DataFrame(anomaly_y)
    random_anomalous_y_copy = random_anomalous_y.copy()
    for row_id in row_ids:
        new_data = [0.0] * len(random_anomalous_y.columns)
        for y in range(len(new_data)):
            if type=="deactivate":
                if (random_anomalous_y.iloc[row_id][y] != 0.0):
                    new_data[y] = np.random.uniform(low=mins[y], high=(maxs[y] - mins[y]))
            elif type=="activate":
                if (random_anomalous_y.iloc[row_id][y] == 0.0):
                    new_data[y] = np.random.uniform(low=mins[y], high= (maxs[y] - mins[y]))
        random_anomalous_y.loc[row_id] = new_data
    plot_anomalsed_data(row_ids, random_anomalous_y, random_anomalous_y_copy, "intentional_anomaly" + type)

In [ ]:
def plot_anomalsed_data(row_ids, dataset, dataset_copy, type):
    colors = ["cornflowerblue","lightsteelblue","mediumblue","blue","slateblue","navy","royalblue", "dodgerblue","red"]
    fig, (ax1, ax2) = pyplot.subplots(1,2, figsize=(17,8), dpi=300, sharey=True)
    ax1.set_xlabel('SEW')
    ax1.set_ylabel('Sensor Value')
    ax2.set_xlabel('SEW')
    ax2.set_ylabel('Sensor Value')
    fig.suptitle("Normal data vs data with random synthetic anomaly injection")
    dataset_copy.plot(color=colors[:-1], ax=ax1)
    anomaly = [0.0] * len(dataset.index)
    for row in row_ids:
        anomaly[row] = dataset.iloc[row].max()
        dataset.loc[row] = [0.0]* NUMBER_PREDICTIONS
    dataset["Anomaly"] = anomaly
    dataset.plot(color=colors, ax=ax2)
    plt.legend(loc = "upper left")
    plt.tight_layout()
    pyplot.savefig("data_plots/" + DATASET + "/" + type)

<a id='section4'></a>
## Section 4: Anomaly Detection Module

This method estimates the likelihood of an anomaly occuring by comparing the predicted value to the ground truth

In [ ]:
def detect_anomaly(anomaly_x, anomaly_y, predictions):
    anomaly_free_x_values = anomaly_x
    anomaly_inserted_y_vales = anomaly_y
    
    for i, prediction in enumerate(predictions):
        anomaly_probability = 0.0
        for x, value in enumerate(prediction):
            anomaly_probability += abs(value - anomaly_inserted_y_vales[i][x])
        print("Probability of anomaly: ", anomaly_probability / len(prediction))

<a id='section5'></a>
## Section 5: Running the system

This section contains the code to run the system on data sets 1 and 2 (note these runs use the final model found for the Prediction Module during hyper parameter tuning)

1. [HH101](#hh101)
2. [HH102](#hh102)

<a id='hh101'></a>

### HH101 

In [393]:
DATASET= "hh101"
# # Data pre-processing 
# hh101, hh101_sensor_id_mapping, hh101_reversed_mapping = load_hh101()
# hh101_drop = ["D002", "MA016", "LS010", "LS009", "LS013", "LS006", "M006", "MA013", "LS015","T101", "T102", "T103", "T104", "T105"]
# hh101 = transform_data(hh101, hh101_sensor_id_mapping)
# # Data pre-processing

# # Feature Selection
# hh101_removed = remove_sensors(hh101, hh101_drop, hh101_sensor_id_mapping)
# most_important_features = pca_decomp(hh101_removed, hh101_reversed_mapping,"hh101")
# # Feature Selection

# # Set up data for prediction module
# hh101_data, plot_data = transform_series_to_supervised(hh101, HH101_PREDICTION_SENSORS, most_important_features, hh101_reversed_mapping)
# NUMBER_IN_FEATURES = len(most_important_features)
# NUMBER_PREDICTIONS = len(HH101_PREDICTION_SENSORS)
# train_x, train_y, test_x, test_y, anomaly_x, anomaly_y = create_train_test_split(hh101_data)
# # Set up data for prediction module

# # Find best hyper-params and train final model, or load best model from file
# tuner = tune_model(train_x, train_y, "hh101")
# final_model = get_final_model(train_x, train_y, test_x, test_y, tuner)
# # Find best hyper-params and train final model, or load best model from file

# numbers = np.row_stack((test_y, train_y, anomaly_y))
# generate_anomalous_data(get_stats(numbers), anomaly_x, anomaly_y)

# plot_normalised_sensor_activations("hh101", plot_data, hh101_reversed_mapping)
plot_cleaned_data(hh101, hh101_reversed_mapping, "hh101", "/cleaned_data")


Load the best model and make predictions

In [ ]:

# %load_ext tensorboard

#   Load best model and make predictions
# model = load_shit()
# predictions = predict_shit()
# Load model and make predictions

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir /Users/christinaspanellis/Desktop/MAC/AAL_ICL/tensorflow/hh102/logs/

In [ ]:
# plot_cleaned_data(hh101, hh101_reversed_mapping, "hh101", "/cleaned_data.png")
# plot_normalised_sensor_activations("hh101", plot_data, hh101_reversed_mapping)

<a id='hh102'></a>

### HH102

In [ ]:
# DATASET = "hh102"
# # Data pre-processing 
# hh102, hh102_sensor_id_mapping, hh102_reversed_mapping = load_hh102()
# hh102_drop = ["LS013", "LS006", "LS011", "M011", "MA010", "LS012", "LS015", "LS009", "LS023", "T101", "T102", "T103", "T104"]
# hh102 = transform_data(hh102, hh102_sensor_id_mapping)
# # Data pre-processing 

# # Feature Selection
# hh102_removed = remove_sensors(hh102, hh102_drop, hh102_sensor_id_mapping)
# most_important_features = pca_decomp(hh102_removed, hh102_reversed_mapping, "hh102")
# # Feature Selection

# # Set up data for prediction module
# hh102_data, plot_data = transform_series_to_supervised(hh102, HH102_PREDICTION_SENSORS, most_important_features, hh102_reversed_mapping)
# NUMBER_IN_FEATURES = len(most_important_features)
# NUMBER_PREDICTIONS = len(HH102_PREDICTION_SENSORS)
# train_x, train_y, test_x, test_y, anomaly_x, anomaly_y = create_train_test_split(hh102_data)
# # Set up data for prediction module

# # This was used for finding the best model, uncomment to see tuner in action
# # model = tune_model(train_x, train_y, "hh102")
# # This was used for finding the best model, uncomment to see tuner in action


Running this cell will create various plots of the data set

In [ ]:
# plot_cleaned_data(hh102, hh102_reversed_mapping, "hh102" , "/cleaned_data.png")
# plot_normalised_sensor_activations("hh102", plot_data, hh102_reversed_mapping)

In [ ]:
# model = tune_model(train_x, train_y, "hh102")